In [1]:
# Импорт библиотек
import os
import time

import GTU
import KU_TU
import mat_properties as prop
import numpy as n
import pandas as pd
import SP
from calculate_CCGT_PKM import Calculate_CCGT_PKM_iter
from scipy.optimize import root

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)
accumulation = pd.read_excel("blocks.xlsx", sheet_name="accumulation", index_col=0)
############################################################
# Теплосеть и перекидка температуры воздуха
gas_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
water_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
Tnv = gas_streams.at["AIR", "T"]
water_streams.at["SWIN", "T"] = SP.Tset(Tnv)[1]
water_streams.at["SWOUT", "T"] = SP.Tset(Tnv)[0]
water_streams.at["SWIN-TURB", "T"] = water_streams.at["SWIN", "T"]
water_streams.at["SWIN-TURB", "G"] = water_streams.at["SWIN", "G"]
water_streams.at["SP2-WOUT", "T"] = water_streams.at["SWOUT", "T"]
############################################################

# Основные эффективности оборудования
KPD_PN = 0.8074
KPD_KN = 0.75
KPD_to = 0.99
KPD_SP = 0.99


# Параметры, отвечающие за процесс расчета
Calcmethod = "hybr"
Calctolerance = 10**-2
Maxiterations_KU_TU = 8
Maxiterations_cotel = 5
Maxiterations_turbine = 15

# Параметры режима работы ПГУ
# Расчет для минимума нагрузки
Сalculate_minimum = True


# Расчет для работы с теплофикацией
# Teplo = int(False)
# Отбор пара высокго давления или доля или кг/с
steamVD_fraction_to_turbine = 1


# Состав газов в номинале в ГТУ
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = list(gas_streams0.loc["GTU-PEVD", "N2":"Ar"])
gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure = gas_streams0.loc["GTU-KU", "P"]
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = list(
    map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures)
)


#####################Максимов#####################
water_streams.at["SWIN", "H"] = water.p_t(1, water_streams.at["SWIN", "T"])["h"]
water_streams.at["SWOUT", "H"] = water.p_t(1, water_streams.at["SWOUT", "T"])["h"]

# Время зарядки/разрядки, часы
time_ac = 4
# Время ожидания, часы
time_jdat = 12
# Конструкция аккумулятора
constr = {
    "Diametr": 20,
    "kolichestvo": 8,
    "Visota": 20,
    "lambda_min_vata": 0.045,
    "delta_min_vata": 0.01,
}
PKM_zaryad = True
PKM_razryad = False
syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
##################################################


############################################################
# Задание ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, "GTU-KU")

Iter_pkm = 4
pkm_pgu_tol = 10**-2

arguments_all_it = [
    Maxiterations_KU_TU,
    Maxiterations_cotel,
    Maxiterations_turbine,
    gas_streams0,
    water_streams0,
    GTU_ISO,
    GTU_input,
    gas_streams,
    water_streams,
    heaters,
    electric,
    Gas_turbine,
    gas0,
    water,
    PKM_zaryad,
    PKM_razryad,
    syngas_streams,
    Calcmethod,
    Calctolerance,
    KPD_PN,
    KPD_KN,
    KPD_to,
    KPD_SP,
    steamVD_fraction_to_turbine,
    accumulation,
    time_ac,
    constr,
    time_jdat,
]


########################ОГРАНИЧЕНИЯ НА РАБОТУ ПГУ########################
# ЕСЛИ ТЕПЛОФИКАЦИЯ (из документов по ПГУ-220Т)
# Gк_мин=4,44 кг/с вроде
# Nтурбины мин = 17,6 МВт (примерно 25%)
# Максимальноа давление в отборах СП 0,245, 0,198 МПа (Теплофикационная паровая турбина Т-63/76-8.8 для серии ПГУ-230)
# ДЛЯ ВСЕХ РЕЖИМОВ (Из Трухния по ПГУ-450Т)
# Gвд_мин=25% от номинала
# Gнд_мин = 50% от номинала


########РАСЧЕТ расхода пара на ПКМ из условия заполнения хранилища################
########РАСЧЕТ мощности ГТУ из условия тепловой мощности и ################

################Расчет минимальной нагрузки ГТУ при остальных нормальных условиях############

Max_iterations_minimum = 20

n_GTU_it = [0]
if Сalculate_minimum == True:
    gas_streams.loc["GTU-PEVD", "G"] = gas_streams.loc["GTU-KU", "G"]
    n_GTU = GTU_input.at["n", 1]
    start_time = time.time()
    Delta_min=0
    n_GTU_it.append(round(n_GTU, 5))
    for i in range(Max_iterations_minimum):
        print("n_GTU:", n_GTU_it)
        if i < 4:
            (
                New_iterations_KU_TU,
                New_iterations_cotel,
                New_iterations_turbine,
                New_Iter_pkm,
            ) = (2, 2, 15, 3)
        else:
            (
                New_iterations_KU_TU,
                New_iterations_cotel,
                New_iterations_turbine,
                New_Iter_pkm,
            ) = (
                Maxiterations_KU_TU,
                Maxiterations_cotel,
                Maxiterations_turbine,
                Iter_pkm,
            )

        gas_streams = Calculate_CCGT_PKM_iter(
            arguments_all_it, New_Iter_pkm, pkm_pgu_tol
        )
        Delt_Gcnd = 100
        Delt_Nturb = 100
        Delt_Gcnd = (water_streams.at["INKOND", "G"] - 4.44) / 4.44
        Delt_Nturb = (electric.at["Turbine", "Ni"] - 17.6) / 17.6
        Delt_Gvd = (
            water_streams.at["PEVD-DROSVD", "G"] / water_streams0.at["PEVD-DROSVD", "G"]
            - 0.25
        ) / 0.25
        Delt_Gnd = (
            water_streams.at["PPND-DROSND", "G"] / water_streams0.at["PPND-DROSND", "G"]
            - 0.5
        ) / 0.5
        
        
        
        arguments_all_it[0], arguments_all_it[1], arguments_all_it[2] = (
            New_iterations_KU_TU,
            New_iterations_cotel,
            New_iterations_turbine,
        )
        
        Delta_n_GTU = abs((n_GTU_it[-1] - n_GTU_it[-2]) / n_GTU_it[-1] * 100)
        Delta_min = min(Delt_Gcnd, Delt_Nturb, Delt_Gvd, Delt_Gnd)
        n_GTU = n_GTU - Delta_min / 3
        GTU_input.at["n", 1] = n_GTU
        n_GTU_it.append(round(n_GTU, 5))
        
        
        print(
            f"Время {i+1} итерации расчета мощности ГТУ при ПГУ с ПКМ: --- {round((time.time() - start_time), 1)} сек. ---"
        )
        print(f"Отклонение от ограничения минимальное равно {Delta_min}")
        
        if n_GTU == 1 and Delta_min < 0:
            print("Мощность ГТУ 100% и расход пара все еще слишком мал")

        if abs(Delta_min) < Calctolerance and Delta_n_GTU < Calctolerance:
            arguments_all_it[0], arguments_all_it[1], arguments_all_it[2] = (
                Maxiterations_KU_TU,
                Maxiterations_cotel,
                Maxiterations_turbine,
            )
            n_GTU_it.append(round(n_GTU, 5))
            gas_streams = Calculate_CCGT_PKM_iter(
                arguments_all_it, New_Iter_pkm, pkm_pgu_tol
            )
            print(f"Отклонение от ограничения минимальное равно {Delta_min}")
            print(f"Относительная мощность ГТУ равна {n_GTU}")
            print(
                f"fin минимальная мощность ПГУ:--- {round((time.time() - start_time), 1)} сек. ---"
            )
            print("n_GTU_it", n_GTU_it)
            break
        if i == Max_iterations_minimum - 1:
            print(
                "Достигнуто максимальное количество итераций минимального расхода в ПГУ",
                i + 1,
            )
            print("n_GTU_it", n_GTU_it)

n_GTU: [0, 1]
Тепла от ПКМ не хватает на теплофикацию. Расход на СП: 65.6
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 1 итерации расчета КУ+ТУ:---  8.1 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 2 итерации расчета КУ+ТУ:---  19.0 сек. ---
Достигнуто максимальное количество итераций давления КУ+ПТУ: 2
Pnd_it [0.77212, 0.59035, 0.48144]
Pvd_it [9.36936, 6.0131, 4.58998]
fin КУ и ТУ:--- 19.0 сек. ---
Время 1 итерации расчета КУ+ТУ с ПКМ: --- 21.4 сек. ---
Тепла от ПКМ не хватает на теплофикацию. Расход на СП: 123.2
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 1 итерации расчета КУ+ТУ:---  7.5 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций давления верх

In [2]:
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,528.514169,0.1,943.007851,366.057295,0.780300,0.123700,0.030100,0.059400,0.006500
REF-SMESH,800.000000,0.1,1531.123730,52.829252,0.710321,0.009967,0.090539,0.180531,0.008642
GTU-PEVD,566.166840,0.1,1017.180016,418.886547,0.771288,0.109054,0.037883,0.074999,0.006776
PEVD-IVD,485.946842,0.1,924.149523,418.886547,0.771288,0.109054,0.037883,0.074999,0.006776
IVD-EVD,263.633561,0.1,675.368482,418.886547,0.771288,0.109054,0.037883,0.074999,0.006776
EVD-PPND,194.651867,0.1,600.713475,418.886547,0.771288,0.109054,0.037883,0.074999,0.006776
PPND-IND,192.515845,0.1,598.418240,418.886547,0.771288,0.109054,0.037883,0.074999,0.006776
IND-GPK,142.921049,0.1,545.373501,418.886547,0.771288,0.109054,0.037883,0.074999,0.006776
GPK-out,88.666944,0.1,487.818298,418.886547,0.771288,0.109054,0.037883,0.074999,0.006776


In [3]:
water_streams

,T,P,H,G,S,X
AIR,0.000000,0.100000,NaN,NaN,NaN,NaN
PEVD-DROSVD,495.108122,3.598348,3439.427079,60.276291,NaN,NaN
IVD-PEVD,254.267540,4.270701,2799.382161,60.276291,NaN,NaN
EVD-IVD,250.382535,4.270701,1087.621360,60.276291,NaN,NaN
PEN-EVD,135.846476,4.270701,573.980180,60.276291,NaN,NaN
BND-PEN,133.870595,0.303076,562.915056,60.276291,NaN,NaN
PPND-DROSND,181.195787,0.300053,2827.166830,9.496312,NaN,NaN
IND-PPND,133.870595,0.303076,2725.351751,9.496312,NaN,NaN
GPK-IND,128.971668,0.303076,542.024696,69.772602,NaN,NaN
GPK-REC,128.971668,0.303076,542.024696,82.131691,NaN,NaN


In [4]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,700.0,2.0,3687.984095,45.441788,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
COOL-HTS,450.0,2.0,3030.242194,45.441788,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
HTS-HTSCOOL,275.0,2.0,2555.053441,45.441788,0.0,0.0,0.086415,0.0,0.513767,0.050042,0.348747,0.001029
HTSCOOL-Separ,100.0,2.0,791.135714,45.441788,0.0,0.0,0.086415,0.0,0.513767,0.050042,0.348747,0.001029
Separ-SGaccum,100.0,2.0,1348.041621,18.074740,0.0,0.0,0.168668,0.0,0.050949,0.097674,0.680700,0.002009
SGaccum-Separacc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Separacc-Sepout,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sepout-COMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sepout-GTU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AIR-COMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
electric

,Ni,N,KPDm,KPD
PEN,0.666965,0.873880,0.763223,0.384568
CVD,NaN,NaN,NaN,NaN
CND,NaN,NaN,NaN,NaN
GTU,NaN,85.065256,NaN,28.293946
DK,NaN,6.130377,NaN,NaN
Turbine,17.602021,NaN,NaN,NaN
Tots1,10.584815,NaN,NaN,0.845765
Tots2,3.655929,NaN,NaN,0.837227
Tots3,1.407940,NaN,NaN,0.809837
Tots4,1.953338,NaN,NaN,0.429938


In [6]:
heaters

,Qw,Qg,KPD
PEVD,38579.529801,38969.222021,0.99
IVD,103168.921077,104211.031391,0.99
EVD,30959.258310,31271.978091,0.99
PPND,951.828534,961.442963,0.99
IND,21997.530139,22219.727413,0.99
GPK,23868.009607,24109.100613,0.99
SP2,6.230556,NaN,0.99
SP1,25.139220,NaN,0.99
OD,1.975923,NaN,0.99
Strans,129587.778607,NaN,NaN


In [7]:
accumulation

,Qw,T,V
ASW,NaN,NaN,NaN
PKM,NaN,250.0,10000.0


In [8]:
PKM_zaryad = False
PKM_razryad = True

Maxiterations_KU_TU = 20
Maxiterations_cotel = 5
Maxiterations_turbine = 20

n_GTU = 1
GTU_input.at["n", 1] = n_GTU

arguments_all_it = [
    Maxiterations_KU_TU,
    Maxiterations_cotel,
    Maxiterations_turbine,
    gas_streams0,
    water_streams0,
    GTU_ISO,
    GTU_input,
    gas_streams,
    water_streams,
    heaters,
    electric,
    Gas_turbine,
    gas0,
    water,
    PKM_zaryad,
    PKM_razryad,
    syngas_streams,
    Calcmethod,
    Calctolerance,
    KPD_PN,
    KPD_KN,
    KPD_to,
    KPD_SP,
    steamVD_fraction_to_turbine,
    accumulation,
    time_ac,
    constr,
    time_jdat,
]
New_Iter_pkm = 20
pkm_pgu_tol = 10**-2
CCGT = Calculate_CCGT_PKM_iter(arguments_all_it, New_Iter_pkm, pkm_pgu_tol)

Тепла в ГВТО хватает на теплофикацию
Не сходится баланс камеры сгорания 41.958396943850374 5.31841237716435e-12 41.95839694384506
Рассчитана пиковая ГТУ, расход воздуха: 936.6890019703188
Время 1 итерации расчета КУ+ТУ:---  10.5 сек. ---
Время 2 итерации расчета КУ+ТУ:---  21.6 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:3
Время 3 итерации расчета КУ+ТУ:---  44.9 сек. ---
Время 4 итерации расчета КУ+ТУ:---  68.2 сек. ---
Время 5 итерации расчета КУ+ТУ:---  80.1 сек. ---
Время 6 итерации расчета КУ+ТУ:---  92.0 сек. ---
Время 7 итерации расчета КУ+ТУ:---  103.2 сек. ---
Время 8 итерации расчета КУ+ТУ:---  120.2 сек. ---
Время 9 итерации расчета КУ+ТУ:---  128.2 сек. ---
Время 10 итерации расчета КУ+ТУ:---  133.6 сек. ---
Время 11 итерации расчета КУ+ТУ:---  137.2 сек. ---
Время 12 итерации расчета КУ+ТУ:---  138.6 сек. ---
Расчет КУ+ПТУ окончен.
Максимальная погрешность определения расходов при расчете КУ+ПТУ 1.8823488591759952e-14
Pnd_it [0.30005, 0.51787, 0.

In [9]:
GTU_input

,1,2,3,4,5,6
tair,0.0,NaN,NaN,NaN,NaN,NaN
n,1.0,NaN,NaN,NaN,NaN,NaN
Pin,0.0,NaN,NaN,NaN,NaN,NaN
Pout,0.0,NaN,NaN,NaN,NaN,NaN


In [10]:
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,528.514169,0.1,943.007851,366.057295,0.780300,0.123700,0.030100,0.059400,0.006500
REF-SMESH,800.000000,0.1,1531.123730,52.829252,0.710321,0.009967,0.090539,0.180531,0.008642
GTU-PEVD,566.166840,0.1,1017.180016,418.886547,0.771288,0.109054,0.037883,0.074999,0.006776
PEVD-IVD,485.946842,0.1,924.149523,418.886547,0.771288,0.109054,0.037883,0.074999,0.006776
IVD-EVD,263.633561,0.1,675.368482,418.886547,0.771288,0.109054,0.037883,0.074999,0.006776
EVD-PPND,194.651867,0.1,600.713475,418.886547,0.771288,0.109054,0.037883,0.074999,0.006776
PPND-IND,192.515845,0.1,598.418240,418.886547,0.771288,0.109054,0.037883,0.074999,0.006776
IND-GPK,142.921049,0.1,545.373501,418.886547,0.771288,0.109054,0.037883,0.074999,0.006776
GPK-out,88.666944,0.1,487.818298,418.886547,0.771288,0.109054,0.037883,0.074999,0.006776


In [11]:
water_streams

,T,P,H,G,S,X
AIR,0.000000,0.100000,NaN,NaN,NaN,NaN
PEVD-DROSVD,506.186547,8.036141,3414.363429,64.015592,NaN,NaN
IVD-PEVD,298.420832,8.398470,2752.619103,64.015592,NaN,NaN
EVD-IVD,297.068789,8.398470,1328.505989,64.015592,NaN,NaN
PEN-EVD,165.607969,8.398470,704.261797,64.015592,NaN,NaN
BND-PEN,164.191140,0.686992,693.710208,64.015592,NaN,NaN
PPND-DROSND,208.809405,0.681200,2865.818810,15.219039,NaN,NaN
IND-PPND,164.191140,0.686992,2761.959140,15.219039,NaN,NaN
GPK-IND,162.354428,0.686992,685.728876,79.234631,NaN,NaN
GPK-REC,162.354428,0.686992,685.728876,89.390055,NaN,NaN


In [12]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,700.000000,2.0,3687.984095,45.441788,0.000000,0.000000,0.063560,0.000000,0.536620,0.050040,0.325900,0.023880
COOL-HTS,450.000000,2.0,3030.242194,45.441788,0.000000,0.000000,0.063560,0.000000,0.536620,0.050040,0.325900,0.023880
HTS-HTSCOOL,275.000000,2.0,2555.053441,45.441788,0.000000,0.000000,0.086415,0.000000,0.513767,0.050042,0.348747,0.001029
HTSCOOL-Separ,100.000000,2.0,791.135714,45.441788,0.000000,0.000000,0.086415,0.000000,0.513767,0.050042,0.348747,0.001029
Separ-SGaccum,100.000000,2.0,1348.041621,18.074740,0.000000,0.000000,0.168668,0.000000,0.050949,0.097674,0.680700,0.002009
SGaccum-Separacc,5.703306,2.0,1075.096789,365775.995050,0.000000,0.000000,0.168738,0.000000,0.050357,0.097508,0.681387,0.002009
Separacc-Sepout,5.703306,2.0,957.542925,336452.977654,0.000000,0.000000,0.177686,0.000000,0.000000,0.102679,0.717520,0.002115
Sepout-COMB,5.703306,2.0,957.542925,22.283679,0.000000,0.000000,0.177686,0.000000,0.000000,0.102679,0.717520,0.002115
Sepout-GTU,5.703306,2.0,957.542925,1.081111,0.000000,0.000000,0.177686,0.000000,0.000000,0.102679,0.717520,0.002115
AIR-COMP,15.000000,0.1,293.187169,936.689002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
electric

,Ni,N,KPDm,KPD
PEN,0.675466,0.700924,0.963680,0.807400
CVD,NaN,NaN,NaN,NaN
CND,NaN,NaN,NaN,NaN
GTU,NaN,164.304600,NaN,34.012097
DK,NaN,6.298505,NaN,NaN
Turbine,74.357899,NaN,NaN,NaN
Tots1,34.504441,NaN,NaN,0.849306
Tots2,9.370963,NaN,NaN,0.837238
Tots3,5.477338,NaN,NaN,0.814967
Tots4,25.005158,NaN,NaN,0.848791


In [14]:
heaters

,Qw,Qg,KPD
PEVD,42364.756329,42792.683160,0.99
IVD,91168.320099,92089.212221,0.99
EVD,39962.075218,40365.732543,0.99
PPND,1580.620437,1596.586300,0.99
IND,32107.044708,32431.358291,0.99
GPK,38794.248256,39186.109350,0.99
SP2,0.000000,NaN,0.00
SP1,0.000000,NaN,0.00
OD,0.000000,NaN,0.00
Strans,0.000000,NaN,NaN


In [15]:
accumulation

,Qw,T,V
ASW,NaN,NaN,NaN
PKM,8.943377e+09,5.703306,10000.0
